In [11]:
import pandas as pd
import os
import re  # Para manejar expresiones regulares

# Configuración
input_file = '4027_VEP_2.csv'  
output_dir = 'resultados'  # Directorio donde se guardarán los resultados

# Crear directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

def filter_variants(row, terms, freq_threshold=0.001):
    """ Filtra variantes con CLIN_SIG que contengan términos específicos y gnomAD ≤ 0.001 """
    if pd.isna(row['Extra']):
        return False
    
    # Buscar si CLIN_SIG contiene alguno de los términos, sin importar el orden
    match = re.search(r'CLIN_SIG=([^;]+)', row['Extra'])  # Captura el valor de CLIN_SIG
    if match:
        clin_sig_values = match.group(1).split(',')  # Divide en lista por comas
        clin_sig_match = any(term in clin_sig_values for term in terms)
    else:
        clin_sig_match = False

    # Buscar la frecuencia en gnomAD dentro de 'Extra'
    freq_match = False
    for entry in row['Extra'].split(';'):
        if entry.startswith("gnomADe_AF="):
            try:
                freq_value = float(entry.split('=')[1])
                if freq_value == 0 or freq_value <= freq_threshold:
                    freq_match = True
            except ValueError:
                pass  # En caso de error, ignorar

    return clin_sig_match and freq_match

# Términos a filtrar
terms_to_filter = ['pathogenic', 'likely_pathogenic', 'uncertain_significance', 'conflicting_interpretations_of_pathogenicity']

# Verificar si el archivo existe
if os.path.exists(input_file):
    # Leer archivo
    df = pd.read_csv(input_file, sep=';', header=0)
    
    # Filtrar variantes
    filtrado = df[df.apply(filter_variants, args=(terms_to_filter,), axis=1)].copy()
    filtrado.loc[:, 'Extra'] = filtrado['Extra'].str.replace('"', '')
    
    if not filtrado.empty:
        # Generar nombres de archivo de salida
        base_name = os.path.splitext(os.path.basename(input_file))[0]
        output_csv = os.path.join(output_dir, f'{base_name}_filtered.csv')
        output_excel = os.path.join(output_dir, f'{base_name}_filtered.xlsx')
        
        # Guardar resultados
        filtrado.to_csv(output_csv, index=False, sep=';')
        filtrado.to_excel(output_excel, index=False)
        
        print(f"Procesado: {input_file} -> {len(filtrado)} variantes guardadas en {output_csv}")
    else:
        print(f"Procesado: {input_file} -> No se encontraron variantes relevantes")
else:
    print(f"Error: El archivo {input_file} no existe.")

print("\nProcesamiento completado. Resultados guardados en:", output_dir)


Procesado: 4027_VEP_2.csv -> 79 variantes guardadas en resultados/4027_VEP_2_filtered.csv

Procesamiento completado. Resultados guardados en: resultados
